## Задача 1. Отличия до эксперимента
  

Возьмите те же группы, что и в эксперименте с изменением дизайна сайта, и проверьте значимость отличий средней выручки с пользователя на неделе перед экспериментом (c 2022.03.16 по 2022.03.23).

Для решения используйте данные из файлов 2022-04-01T12_df_sales.csv и experiment_users.csv.

В качестве ответа введите p-value, округлённое до 3-го знака после точки.

In [26]:
import pandas as pd
import scipy.stats as stats

In [11]:
sales_df = pd.read_csv("2022-04-01T12_df_sales.csv")
exp_df = pd.read_csv("experiment_users.csv")
sales_df['date'] = pd.to_datetime(sales_df['date'])

In [4]:
sales_df.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [7]:
exp_df.head()

,user_id,pilot
0,0ffc65,0
1,b962b9,0
2,7ea63f,0
3,7f9a61,0
4,459e55,0


In [76]:
exp_period = pd.date_range("2022-03-16", "2022-03-23")

In [87]:
sales_df = sales_df[(sales_df['date'] >= exp_period[0]) & (sales_df['date'] < exp_period[-1])]\
    .groupby("user_id", as_index=False)['price'].sum()

In [89]:
sales_df

,user_id,price
0,000096,720
1,00092c,780
2,000bb2,720
3,000ea9,1560
4,000ec6,690
...,...,...
24595,fff3ec,660
24596,fff6b5,3480
24597,fff8bf,2880
24598,fff98d,3480


In [96]:
control_revenue = exp_df[exp_df['pilot'] == 0].merge(sales_df, how='left')['price'].fillna(0)

In [97]:
pilot_revenue = exp_df[exp_df['pilot'] == 1].merge(sales_df, how='left')['price'].fillna(0)

In [98]:
result = stats.ttest_ind(control_revenue, pilot_revenue)

In [99]:
round(result.pvalue, 3), result.statistic

(np.float64(0.199), np.float64(-1.2837567415000515))

## Задача 2. Среднее время между покупками
  

При выполнении прошлого задания вы могли заметить, что в данных много нулей. Это значит, что большая часть пользователей, совершивших покупку во время эксперимента, не совершала покупок на неделе до эксперимента. Интересно, как часто наши клиенты делают покупки?

Оцените среднее время между покупками. Возьмите всех клиентов, у которых 2 и более покупок. Вычислите время между покупками (для клиента с N покупками должно получиться N-1 значения времени). Объедините значения всех клиентов и вычислите среднее.

Для решения используйте данные из файлов 2022-04-01T12_df_sales.csv.

В качестве ответа введите среднее количество дней между покупками, округлённое до целого значения.

In [102]:
import pandas as pd

In [103]:
sales_df = pd.read_csv("2022-04-01T12_df_sales.csv")

In [104]:
sales_df.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [105]:
sales_df['date'] = pd.to_datetime(sales_df['date'])

In [111]:
time_diff = sales_df.sort_values(["user_id", "date"]).groupby('user_id')["date"].diff()

In [113]:
time_diff[~time_diff.isna()].mean()

Timedelta('17 days 07:37:57.653431857')

## Задача 3. Функция для проверки статистической значимости
 

Для проверки статистической значимости нужно вычислить p-value и сравнить его с уровнем значимости.

Напишите функцию get_ttest_pvalue. 

Шаблон решения

In [116]:
import numpy as np
from scipy import stats


def get_ttest_pvalue(metrics_a_group, metrics_b_group):
    """Применяет тест Стьюдента, возвращает pvalue.

    :param metrics_a_group (np.array): массив значений метрик группы A
    :param metrics_a_group (np.array): массив значений метрик группы B
    :return (float): значение p-value
    """
    result = stats.ttest_ind(metrics_a_group, metrics_b_group)
    return result.pvalue

In [117]:
metrics_a_group = np.array([964, 1123, 962, 1213, 914, 906, 951, 1033, 987, 1082])
metrics_b_group = np.array([952, 1064, 1091, 1079, 1158, 921, 1161, 1064, 819, 1065])
pvalue = get_ttest_pvalue(metrics_a_group, metrics_b_group)

In [118]:
pvalue

np.float64(0.6122191629541949)